In [1]:
!pip install langchain_core -q
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device='cuda' if torch.cuda.is_available() else 'cpu'

model_name='tutran27/Chat_bot_Qwen3-4B'

model=AutoModelForCausalLM.from_pretrained(model_name,
                                           trust_remote_code=True,
                                           device_map=device,
                                           dtype=torch.float16)

tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [7]:
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

pipe_model=pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1,
    return_full_text=False,
)

llms=HuggingFacePipeline(pipeline=pipe_model)

Device set to use cuda
/tmp/ipython-input-2626427090.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llms=HuggingFacePipeline(pipeline=pipe_model)


In [17]:
input=""" <|im_start|> Viết 1 đoạn ngắn sánh chó và mèo <|im_end|>
          <|im_start|>"""

print(llms.invoke(input))



Chó và mèo là hai loài động vật có vú được coi là bạn đồng hành trong nhiều thế kỷ. Dù chúng có những đặc điểm khác nhau nhưng cả hai đều có một số điểm chung. Cả chó và mèo đều được coi là thú cưng và chúng thường gắn bó chặt chẽ với con người. Chúng cũng có thể rất thông minh, trung thành và dễ nuôi. Mặc dù chúng có cách tiếp cận cuộc sống khác nhau nhưng cả hai đều có thể trở thành thành viên gia đình tuyệt vời của bất kỳ ai có lòng tốt đối xử với chúng.


# Load document

In [24]:
!pip install pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 11.6 MB/s eta 0:00:00


In [25]:
from langchain_community.document_loaders import PyPDFLoader

data=PyPDFLoader("/content/Lý thuyết.pdf")
document=data.load()

In [ ]:
print(document[0])

# Splittext

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

split_text=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=40,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

texts=split_text.split_documents(document)

In [ ]:
for x in texts:
  print(x)
  break

# Embedding

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_name='Qwen/Qwen3-Embedding-0.6B'
embedding_function=HuggingFaceEmbeddings(model_name=embedding_name)

# Vector Database

In [ ]:
!pip install chromadb -q

In [40]:
from langchain_community.vectorstores import Chroma

db=Chroma.from_documents(texts, embedding_function)

# Retriever

In [60]:
retriever=db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

query="Sóng dừng là gì"
output=retriever.invoke(query)
print(output[1].page_content)

Ω ≈ 𝜔0 
 
(với 𝜔0là tần số góc riêng của hệ; ma sát càng nhỏ → cộng hưởng càng rõ) 
 
CHƯƠNG II: SÓNG 
1) Mô tả sóng cơ 
• Sóng cơ: sự lan truyền dao động cơ trong môi trường vật chất.


In [45]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [52]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

str_parser=StrOutputParser()

template=""" <|im_start|> Bạn là một chatbot truy xuất thông tin rất hữu ích. Từ thông tin được cung cấp hãy hoàn thiện câu trả lời một cách chính xác. <|im_end|>
<|im_start|> {context} <|im_end|>
<|im_start|> Câu hỏi: {question} <|im_end|>
<|im_start|>"""

prompt=PromptTemplate( template=template)


In [53]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llms
    | str_parser
)

In [56]:
output=rag_chain.invoke("Giải thích sóng ngang")
print(output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Wavengang là những sóng có phương dao động vuông góc với hướng chuyển động của chúng. Điều này có nghĩa là khi các hạt trong vật chất dao động theo phương thẳng đứng hoặc nằm ngang, chúng chuyển động theo chiều ngang hay dọc so với hướng truyền năng lượng. Ví dụ về sóng ngang bao gồm sóng trên mặt nước và sóng cơ học trong các sợi dây như dây đàn gõ. Trong sóng ngang, các hạt vật chất di chuyển theo hướng vuông góc với hướng truyền năng lượng. Các sóng ngang cũng được gọi là sóng transverse vì chúng liên quan đến chuyển động của các hạt vật chất trong vật liệu mà chúng đang truyền năng lượng. Nguồn phát ra sóng ngang có thể là bất kỳ nguồn nào tạo ra dao động trên bề mặt của một vật liệu nhất định, chẳng hạn như một dòng chảy trong nước hoặc sự rung động của dây. Nhìn chung, sóng ngang là loại sóng phổ biến nhất được tìm thấy trong môi trường tự nhiên, bao gồm cả các hiện tượng tự nhiên như sóng biển và sóng trên băng biển. Chúng cũng thường được sử dụng trong các ứng dụng công nghiệp